In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import imageio
import os
import time
import sys

from liblibra_core import *
import libra_py.models.Holstein as Holstein
import util.libutil as comn
from libra_py import data_outs

import libra_py.dynamics.qtag.qtag_basis as qtag_basis
import libra_py.dynamics.qtag.qtag_calc as qtag_calc
import libra_py.dynamics.qtag.qtag_checks as qtag_checks
import libra_py.dynamics.qtag.qtag_ham as qtag_ham
import libra_py.dynamics.qtag.qtag_init as qtag_init
import libra_py.dynamics.qtag.qtag_mom as qtag_mom
import libra_py.dynamics.qtag.qtag_prop as qtag_prop


/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/alexey/Conda/Miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::v

In [2]:
class tmp:
    pass

#Holstein5 from libra_py/models/Hostein.py
def holstein_mimic(q, params, full_id):
    critical_params = ["E_n", "x_n", "k_n" ]
    default_params = { "V": [ [0.001, 0.001, 0.001, 0.001], [0.001, 0.001, 0.001, 0.001],
                              [0.001, 0.001, 0.001, 0.001], [0.001, 0.001, 0.001, 0.001] ],
                       "alpha": [ [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00],
                                  [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00] ],
                       "x_nm": [ [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00],
                                 [0.00, 0.00, 0.00, 0.00], [0.00, 0.00, 0.00, 0.00] ],
                     }
    comn.check_input(params, default_params, critical_params)

    E_n = params["E_n"]
    x_n = params["x_n"]
    k_n = params["k_n"]
    V = params["V"]
    alpha = params["alpha"]
    x_nm = params["x_nm"]
    
    n = len(E_n)

    Hdia = CMATRIX(n,n)
    Sdia = CMATRIX(n,n)
    d1ham_dia = CMATRIXList();  d1ham_dia.append( CMATRIX(n,n) )
    dc1_dia = CMATRIXList();  dc1_dia.append( CMATRIX(n,n) )

    
    Id = Cpp2Py(full_id)
    indx = Id[-1]
    
    #print(indx)

    x = q.col(indx).get(0)

    
    #indx = 0
    #x = q.col(indx).get(0)

    Sdia.identity()

    for i in range(n):
        Hdia.set(i,i,  (E_n[i] + 0.5*k_n[i]*(x - x_n[i])**2) * (1.0+0.0j) )

    for i in range(n):
        for j in range(n):
            if i!=j:
                Hdia.set(i,j,  V[i][j] * math.exp(-alpha[i][j] * (x-x_nm[i][j])**2 ) * (1.0+0.0j) )

    for k in [0]:
        #  d Hdia / dR_0
        for i in range(n):
            d1ham_dia[k].set(i,i, (k_n[i] * (x - x_n[i]))*(1.0+0.0j) )

    for k in [0]:
        for i in range(n):
            for j in range(n):
                if i!=j:
                    d1ham_dia[k].set(i,j,  -2.0*alpha[i][j] * (x-x_nm[i][j]) * V[i][j] * math.exp(-alpha[i][j] * (x-x_nm[i][j])**2 ) * (1.0+0.0j) )

    obj = tmp()
    obj.ham_dia = Hdia
    obj.ovlp_dia = Sdia
    obj.d1ham_dia = d1ham_dia
    obj.dc1_dia = dc1_dia

    return obj

In [3]:
def compute_model(q, model_params, full_id):

    model = model_params["model"]
    res = None
    
    if model==1:
        #res = Holstein.Holstein2(q, model_params, full_id)
        res = holstein_mimic(q, model_params, full_id)
    else:
        print("Unrecognized model! Check for correct model import statement.")

    return res

In [4]:
def run_qtag(dyn_params, _compute_model, _model_params, prefix):

    # THESE CHECKS SHOULD BE DONE IN CORRESPONDING FUNCTIONS

    #qtag_checks.user_input(dyn_params, dyn_params, model_params)
    
    #Define function objects to be used in various parts of the calculation...
    #initialize, basis_props, vapprox, mom_calc, propagate = qtag_checks.assign_fobj(qtag_params)

    basis_props = qtag_checks.set_basis_updates(dyn_params)
    
    #Rename variables locally for convenience...
    ndof = dyn_params['ndof']
    nstates = dyn_params["nstates"]
    active_state = dyn_params["active_state"]
    niter = dyn_params['niter']
    dt = dyn_params['dt']
    mass = dyn_params['mass']
    iM = dyn_params["iM"]

    #Initialize the basis parameters {q,p,a,s} and create initial b-matrix...
    ntraj, qpas = qtag_init.initialize(dyn_params)    

    #Create initial projection vector b...
    bt = qtag_init.coeffs(dyn_params, qpas, active_state)

    #Start simulation and walltime variables...
    walltime_start = time.time()
    t=0.0

    
    ham = nHamiltonian(nstates, nstates, ndof)
    ham.add_new_children(nstates, nstates, ndof, ntraj)
    ham.init_all(2,1)
    model_params.update({"timestep":0})

    ovlp = CMATRIX(ntraj, ntraj)
    hmat = CMATRIX(ntraj, ntraj)
    
    coeff = CMATRIX(nstates, ntraj)
    
        
    #ham.compute_diabatic(_compute_model, qpas[0], _model_params, 1)
    
    prms = dyn_control_params()
    prms.set_parameters(dyn_params)
    
    
    for iter in range(niter):
        
        
        #Calculate system overlap on each state, placed into a super-matrix ovlp...
        #ovlp = qtag_calc.super_overlap(ndof,ntraj,nstates,qpas)

        #Calculate system Hamiltonian for all states and couplings, placed into a super-matrix hmat...
        #hmat = qtag_ham.super_hamiltonian(ndof,ntraj,nstates,mass,qpas, \
        #                                  dyn_params["qtag_pot_approx_method"], \
        #                                  _compute_model, _model_params)
        
        qtag_hamiltonian_and_overlap(qpas[0], qpas[1], qpas[2], qpas[3], coeff, 
                                     Py2Cpp_int(qpas[4]), iM, ham,
                                     _compute_model, _model_params, dyn_params,
                                     ovlp, hmat)
        
        #data_outs.print_matrix(ovlp)
        #data_outs.print_matrix(hmat)
        
        #sys.exit(0)
        
        #Compute the new coefficient vector c_new...
        ct_new = qtag_ham.basis_diag(ntraj,dt,hmat,ovlp,bt)
        
        #New way for coefficient vector c_new...
#        ct_new = qtag_ham.basis_diag(ntraj,dt, super_ham, super_ovlp, bt)
            
        etot = qtag_calc.energy(ct_new,hmat)
        pops = qtag_calc.norm(qpas[4],ct_new,ovlp,
                              [state for state in range(nstates)])

        print(etot)
        #Update the basis parameters according to the new wavefunction (ct_new)...
        #qpasn,bt=qtag_prop.propagate(dyn_params, dyn_params, \
        #                             basis_props,qpas,ct_new,pops)
        
        qpasn,bt=qtag_prop.propagate(dyn_params, basis_props,qpas,ct_new,pops)

    print(pops)
    walltime_end = time.time()
    print("Total wall time: ",walltime_end-walltime_start)


In [5]:
#Parameters for general simulation
iM = MATRIX(1, 1);  iM.set(0,0, 1.0);

dyn_params = {
    "ndof" : 1, "nstates" : 2, "active_state" : 0,
    "dt":0.01, "niter":50, "mass":[1.0],
    "nsnapshots":1, "noutput":10,    
    "init_placement":0, "grid_dims" : [25], "alp_scl":[6.0], "rho_cut":1e-12,
    "basis_qtype" : 1, "basis_ptype" : 1, "basis_atype" : 1, "basis_stype" : 0,
    "wfc_q0" : [-2.0], "wfc_p0" : [0.0], "wfc_a0" : [1.0], "wfc_s0" : [0.0],
    "prop_method" : 1, "decpl_den" : 0.3,
    "mom_calc_type" : 1, "linfit_beta" : 1e-3,
    "qtag_pot_approx_method":0,  "iM":iM        
}


#QTAG-specific parameters, mostly function objects
#qtag_params = {
#    "traj0":{"placement":"grid", "grid_dims" : [25], "a0":[6.0], "rho":1e-12},
#    "basis":{"qtype" : "adpt", "ptype" : "adpt", "atype" : "adpt", "stype" : "frzn"},
#    "wf0":{"q" : [-2.0], "p" : [0.0], "a" : [1.0], "s" : [0.0]},
#    "mss":{"prop_method" : "sync", "decpl" : 0.3, "mirror" : "true"},
#    "mom_params":{"adjust" : "lin_fit", "beta" : 1e-3},
#    "v_approx":'BAT',
#    "qtag_pot_approx_method":0
#        
#}

#Potential parameters for Holstein5
model_params = {
    "model":1,
    "E_n":[0.0,15.8114],
    "x_n":[0.0,1.0],
    "k_n":[10.0,10.0],
    "V":[[0.0,1.0],[1.0,0.0]],
    "alpha":[[0.00,1.5811],[1.5811,0.00]],
    "x_nm":[[0.0,2.0],[2.0,0.0]]
}

run_qtag(dyn_params, compute_model, model_params, "qtag")

19.55555555429452
19.55555555361537
19.555555552760612
19.555555552346313
19.55555555232017
19.55555555371161
19.55555555471342
19.55555555656188
19.55555555836113
19.555555559893207
19.555555561802493
19.555555563356585
19.555555565165538
19.555555566511234
19.555555567947142
19.55555556894414
19.55555556975295
19.555555570332455
19.555555570967215
19.55555557132634
19.555555571531716
19.555555571607083
19.555555571749
19.55555557178535
19.555555571829817
19.55555557207657
19.55555557197405
19.555555571857546
19.555555571805538
19.555555571610128
19.555555571486035
19.555555571942918
19.555555571507846
19.55555557154738
19.555555571526938
19.55555557119666
19.55555557138726
19.55555557139991
19.55555557131001
19.555555571189036
19.555555571109934
19.555555571388854
19.55555557194518
19.555555572804337
19.55555557389291
19.55555557596744
19.555555578306762
19.555555581293284
19.55555558426612
19.55555558779352
[0.999966919608837, 3.308236818830181e-05]
Total wall time:  2.1557810306549